In [1]:
!pip install transformers datasets --quiet

import os
os.environ["WANDB_DISABLED"] = "true"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from datasets import load_dataset
import pandas as pd
import torch

In [3]:
if torch.cuda.is_available():
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA not available. Using CPU.")

Using GPU: Tesla T4


In [4]:
dataset_path = "tamil_slang_large_dataset.tsv"
data = pd.read_csv(dataset_path, sep="\t", header=0, names=["Text", "Normalized Text"])

train_data = data.sample(frac=0.8, random_state=42)
val_data = data.drop(train_data.index)
train_data.to_json("train.json", orient="records", lines=True)
val_data.to_json("valid.json", orient="records", lines=True)

In [5]:
dataset = load_dataset("json", data_files={"train": "train.json", "validation": "valid.json"})

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [6]:
tokenizer = AutoTokenizer.from_pretrained("facebook/mbart-large-50")

def preprocess_function(examples):
    inputs = tokenizer(
        examples["Text"], max_length=32, truncation=True, padding="max_length"
    )
    targets = tokenizer(
        examples["Normalized Text"], max_length=32, truncation=True, padding="max_length"
    )
    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [7]:
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/mbart-large-50").to("cuda")

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [8]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
    save_total_limit=2,
    save_steps=500,
    logging_dir="./logs",
    report_to="none",
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
)

<ipython-input-9-8a438e11e626>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [10]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.000030
2,3.640100,0.000013
3,3.640100,0.000011


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=750, training_loss=2.4267450989658634, metrics={'train_runtime': 922.177, 'train_samples_per_second': 26.025, 'train_steps_per_second': 0.813, 'total_flos': 1625347325952000.0, 'train_loss': 2.4267450989658634, 'epoch': 3.0})

In [11]:
model.save_pretrained("./tamil_slang_mbart")
tokenizer.save_pretrained("./tamil_slang_mbart")

('./tamil_slang_mbart/tokenizer_config.json',
 './tamil_slang_mbart/special_tokens_map.json',
 './tamil_slang_mbart/sentencepiece.bpe.model',
 './tamil_slang_mbart/added_tokens.json',
 './tamil_slang_mbart/tokenizer.json')

In [12]:
def normalize_text(input_text, model, tokenizer):
    inputs = tokenizer(input_text, return_tensors="pt", max_length=32, truncation=True).to("cuda")
    outputs = model.generate(inputs["input_ids"])
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [13]:
test_text = "சரி செம்ம"
normalized_output = normalize_text(test_text, model, tokenizer)
print(f"Normalized Output: {normalized_output}")

Normalized Output: மிகவும் நல்லது


In [14]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_dir = "./tamil_slang_mbart"
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(model_dir)

def normalize_text(input_text, model, tokenizer):
    inputs = tokenizer(input_text, return_tensors="pt", max_length=32, truncation=True).to("cuda")
    outputs = model.generate(inputs["input_ids"])
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

sample_texts = [
    "சரி செம்ம",
    "அப்போ செம்ம பாஸ்",
    "ஆமா கண்டிப்பா சரியா",
    "ஆமா அய்யோ டா",
    "சந்தோஷம் பண்ணிடுவோம்",
    "கேவலமாக சரியா"
]

for slang in sample_texts:
    normalized = normalize_text(slang, model, tokenizer)
    print(f"Input: {slang} -> Normalized: {normalized}")


Input: சரி செம்ம -> Normalized: மிகவும் நல்லது
Input: அப்போ செம்ம பாஸ் -> Normalized: மிகவும் நல்லது
Input: ஆமா கண்டிப்பா சரியா -> Normalized: மிகவும் உறுதியாக
Input: ஆமா அய்யோ டா -> Normalized: விசமம்
Input: சந்தோஷம் பண்ணிடுவோம் -> Normalized: மகிழ்ச்சி நாங்கள் செய்வோம்
Input: கேவலமாக சரியா -> Normalized: மிகவும் வலமாக


In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

model_dir = "Rajkumar57/TamilSlangNormalization"  
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir).to("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(model_dir)

def normalize_text(input_text, model, tokenizer):
    
    inputs = tokenizer(input_text, return_tensors="pt", max_length=32, truncation=True).to(model.device)
    
    outputs = model.generate(inputs["input_ids"])

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

sample_texts = [
    "சரி செம்ம",                
    "அப்போ செம்ம பாஸ்",          
    "ஆமா கண்டிப்பா சரியா",        
    "ஆமா அய்யோ டா",             
    "சந்தோஷம் பண்ணிடுவோம்",      
    "கேவலமாக சரியா"             
]

# Predict the normalized text for each slang
for slang in sample_texts:
    normalized = normalize_text(slang, model, tokenizer)
    print(f"Input: {slang} -> Normalized: {normalized}")

config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

c:\Users\lenovo\anaconda3\envs\myenv\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\lenovo\.cache\huggingface\hub\models--Rajkumar57--TamilSlangNormalization. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/992 [00:00<?, ?B/s]

Input: சரி செம்ம -> Normalized: மிகவும் நல்லது
Input: அப்போ செம்ம பாஸ் -> Normalized: மிகவும் நல்லது
Input: ஆமா கண்டிப்பா சரியா -> Normalized: மிகவும் உறுதியாக
Input: ஆமா அய்யோ டா -> Normalized: விசமம்
Input: சந்தோஷம் பண்ணிடுவோம் -> Normalized: மகிழ்ச்சி நாங்கள் செய்வோம்
Input: கேவலமாக சரியா -> Normalized: மிகவும் வலமாக
